In [12]:
# Library imports
import numpy as np
import modern_robotics as mr
import csv

R = 0.0475 # meters
W = 0.15 # meters
L = 0.47/2 # meters

In [13]:
def NextState(curr_state: np.ndarray, velocities: np.ndarray, dt: float, max_vel: float) -> np.ndarray:
    next_state = np.zeros(13)

    # determine the new joint angles
    next_state[3:8] = curr_state[3:8] + dt * velocities[:5]

    # determine the new wheel angles
    next_state[8:12] = curr_state[8:12] + dt * velocities[5:]

    # calculate the change in wheel angles
    d_wheel_angles = next_state[8:12] - curr_state[8:12]

    # eqn 13.10 (pg. 541)
    H_0 =  (1/R) * np.array([[-L-W, 1, -1], [L+W, 1, 1], [L+W, 1, -1], [-L-W, 1, 1]])

    # body twist is a 3x1 vector
    # eqn 13.33 (pg. 569)
    #   w_bz
    #   v_bx
    #   v_by
    body_twist = np.matmul(np.linalg.pinv(H_0), d_wheel_angles)

    # eqn 13.35 (pg. 570)
    dq_b = np.zeros(3)
    if body_twist[0] == 0.0:
        dq_b = body_twist
    else:
        dq_b = np.array([
            body_twist[0],
            ( body_twist[1] * np.sin(body_twist[0]) + body_twist[2] * (np.cos(body_twist[0]) - 1) ) / body_twist[0],
            ( body_twist[2] * np.sin(body_twist[0]) + body_twist[1] * (1 - np.cos(body_twist[0])) ) / body_twist[0]
        ])
    
    # eqn 13.36 (pg 570)
    dq = np.matmul(np.array([
        [1, 0, 0],
        [0, np.cos(curr_state[0]), -np.sin(curr_state[0])],
        [0, np.sin(curr_state[0]), np.cos(curr_state[0])],
    ]), dq_b)

    next_state[:3] = curr_state[:3] + dq

    return next_state    

In [14]:
def TrajectoryGenerator(T_se_i, T_sc_i, T_sc_f, T_ce_grasp, T_ce_standoff, k: int, Tf: int):
    # number of configurations
    N = int( (Tf * k) / 0.001 )
    output = [() for _ in range(6 * N + 2)]
    idx = 0

    # Trajectory to standoff position
    T_se_standoff_pick = np.matmul(T_sc_i, T_ce_standoff)
    traj = mr.ScrewTrajectory(T_se_i, T_se_standoff_pick, Tf, N, 5)
    for t in traj:
        output[idx] = (t, 0)
        idx += 1

    # Trajectory to grasp position
    T_se_grasp_pick = np.matmul(T_sc_i, T_ce_grasp)
    traj = mr.ScrewTrajectory(T_se_standoff_pick, T_se_grasp_pick, Tf, N, 5)
    for t in traj:
        output[idx] = (t, 0)
        idx += 1

    # open the gripper
    output[idx] = (T_se_grasp_pick, 1)
    idx += 1

    # move back to the standoff point for the block pick up
    traj = mr.ScrewTrajectory(T_se_grasp_pick, T_se_standoff_pick, Tf, N, 5)
    for t in traj:
        output[idx] = (t, 1)
        idx += 1

    # move to the standoff point for the block drop off
    T_se_standoff_drop = np.matmul(T_sc_f, T_ce_standoff)
    traj = mr.ScrewTrajectory(T_se_standoff_pick, T_se_standoff_drop, Tf, N, 5)
    for t in traj:
        output[idx] = (t, 1)
        idx += 1

    # place down the block
    T_se_grasp_drop = np.matmul(T_sc_f, T_ce_grasp)
    traj = mr.ScrewTrajectory(T_se_standoff_drop, T_se_grasp_drop, Tf, N, 5)
    for t in traj:
        output[idx] = (t, 1)
        idx += 1
    
    # open the gripper
    output[idx] = (T_se_grasp_drop, 0)
    idx += 1

    # move the the standoff point above the drop location
    traj = mr.ScrewTrajectory(T_se_grasp_drop, T_se_standoff_drop, Tf, N, 5)
    for t in traj:
        output[idx] = (t, 0)
        idx += 1

    return output

In [15]:
def FeedbackControl(X, X_d, X_d_next, K_p, K_i, delt, joint_angles):
    l = 0.47/2
    w = 0.3/2
    r = 0.0475
    error = np.zeros((6,1))
    
    phi = joint_angles[0]
    x = joint_angles[1]
    y = joint_angles[2]
    
    F = r/4 * np.asarray([[-1/(l+w), 1/(l+w), 1/(l+w), -1/(l+w)], [1, 1, 1, 1], [-1, 1, -1, 1]])
    
    F6 = np.zeros((6,4))
    F6[2:5, :] = F
    
    M = np.asarray([[1, 0, 0, 0.033], [0, 1, 0, 0], [0, 0, 1, 0.6546], [0, 0, 0, 1]])

    T_b0 = np.asarray([[1, 0, 0, 0.1662], [0, 1, 0, 0], [0, 0, 1, 0.0026], [0, 0, 0, 1]])
    
    T_sb = np.asarray([[np.cos(phi), -np.sin(phi), 0, x], [np.sin(phi), np.cos(phi), 0, y], [0, 0, 1, 0.0963], [0, 0, 0, 1]])
    
    Vd = mr.se3ToVec(1/delt * mr.MatrixLog6(mr.TransInv(X_d) @ X_d_next)).reshape((6,1))
    Xerr = mr.se3ToVec(mr.MatrixLog6(mr.TransInv(X) @ X_d)).reshape((6,1))
    ad = mr.Adjoint(mr.TransInv(X) @ X_d)
    error += Xerr*delt
    
    Blist = np.asarray([[0, 0, 1, 0, 0.033, 0], [0, -1, 0, -0.5076, 0, 0], 
                        [0, -1, 0, -0.3526, 0, 0], [0, -1, 0, -0.2176, 0, 0], 
                       [0, 0, 1, 0, 0, 0]]).T
    
    Jarm = mr.JacobianBody(Blist, joint_angles[3:])
    
    T_0e = mr.FKinBody(M, Blist, joint_angles[3:])
    
    ad2 = mr.Adjoint(mr.TransInv(T_0e)@mr.TransInv(T_b0))
    
    Jbase = ad2 @ F6
    
    J = np.concatenate((Jbase,Jarm), axis=1)
    
    V = ad @ Vd + K_p @ Xerr + K_i @ error

    # Calculate the new end effector position (pg. 570)
    T_se = T_sb @ T_b0 @ T_0e
    
    output = np.linalg.pinv(J) @ V
    return (output, Xerr, T_se)

In [16]:
def FullProgram() -> None:
    """
        T_ci: SE(6) Initial Resting Configuration of the cube frame
        T_cd: SE(6) Desired Configuration of the cube frame
        T_ri: SE(6) Initial configuration of the robot
        T_ref: SE(6) Reference Initial Configuration
        K_p: float Proportional gain
        K_i: float Integral Gain
    """

    steps = []
    err_v_time = []

    # T_se = np.array([[0.170, 0, 0.985, 0.387], [0, 1, 0, 0], [-0.985, 0, 0.170, 0.570], [0, 0, 0, 1]])
    T_se = np.array([[0, 0, 1, 0], [0, 1, 0, 0], [-1, 0, 0, 0.5], [0, 0, 0, 1]])
    T_sc_i = np.array([[1, 0, 0, 1], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
    T_sc_f = np.array([[np.cos(-np.pi/2), -np.sin(-np.pi/2), 0, 0], [np.sin(-np.pi/2), np.cos(-np.pi/2), 0, -1], [0, 0, 1, 0], [0, 0, 0, 1]])
    T_ce_grasp = np.array([[-1, 0, 0, 0], [0, 1, 0, 0], [0, 0, -1, 0.005], [0, 0, 0, 1]])
    T_ce_standoff = np.array([[-1, 0, 0, 0], [0, 1, 0, 0], [0, 0, -1, 0.1], [0, 0, 0, 1]])

    K_p = np.zeros((6,6))
    K_i = np.zeros((6,6))

    k = 1 # number of steps per 0.001 seconds
    Tf = 1 # trajectory runtime (s)
    dt = 0.01

    trajectory = TrajectoryGenerator(T_se, T_sc_i, T_sc_f, T_ce_grasp, T_ce_standoff, k, Tf)
    curr_state = np.zeros(12)

    for idx, traj in enumerate(trajectory):
        step, gripper_state = traj
        if idx == len(trajectory) - 1:
            break
        
        T_se_d = step
        T_se_d_next, _ = trajectory[idx + 1]
        velocities, err, T_se = FeedbackControl(T_se, T_se_d, T_se_d_next, K_p, K_i, dt, np.array(curr_state[:8]))

        curr_state = NextState(curr_state, velocities.reshape(1, 9)[0], dt, 20)

        curr_state[12] = gripper_state
        steps.append(curr_state)
        err_v_time.append(err)
    
    with open("final.csv", "w+") as file:
        writer = csv.writer(file)

        writer.writerows(steps)

FullProgram()